In [2]:
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat

from Toolbox_GCSR.Preprocessing import Processor
from sklearn.preprocessing import minmax_scale
from Toolbox_GCSR.utility import eval_band, eval_band_cv
from Toolbox_GCSR.EGCSR_BS_Ranking import EGCSR_BS_Ranking

from Toolbox_bombs.runner import main
from Toolbox_bombs.utils import Arguments

import matlab.engine
eng = matlab.engine.start_matlab()

eng.addpath(eng.genpath("Toolbox_SVD"));
eng.addpath(eng.genpath("Toolbox_OCF"));

Data Importation and adequation

In [3]:
root = 'D:\Python\HSI_Files\\'
im_, gt_ = 'CornData', 'CornLabels'

img_path = root + im_ + '.mat'
gt_path = root + gt_ + '.mat'

p = Processor()
data, labels = p.prepare_data(img_path, gt_path)



Data importation matlab

In [ ]:
data_m = eng.double(eng.importdata(img_path))

Parameters and variables

In [11]:
b_num = 8


Algoritmo SVD

In [ ]:
select_svd = eng.svd_function(data_m, b_num);

b = np.array(select_svd)

print(b.reshape(1,b_num))
print(np.delete(b.reshape(1,b_num),0))

Algoritmo OCF

In [15]:


select_ocf = eng.ocf_trc_fdpc(data_m, b_num);

b = np.array(select_svd)

print(b.reshape(1,b_num))
print(np.delete(b.reshape(1,b_num),0))

Algoritmo bombs

In [ ]:
RESULTS_DIR = os.path.join(
    os.getenv("Bombs_results", os.path.join("..", "..", "results")),
    "bombs_band_selection"
)

arguments = Arguments(
        bands_per_antibody=(i+1)*factor,
        #Data
        data_path=os.path.join(DATA_DIR_CORN, im_),
        ref_map_path=os.path.join(DATA_DIR_CORN, gt_),
        dest_path=RESULTS_DIR,
        Gmax=6,
        Na=10,
        Nd=25,
        Nc=25,
        TD_size=30,
        P_init_size=100,
    )
bands_bombs[i,:] = main(args=arguments)

Algoritmo GCSR

In [ ]:
alg_Ranking = EGCSR_BS_Ranking((i+1)*factor, regu_coef=1e2, n_neighbors=3, ro=0.8)
selected_img = np.array(alg_Ranking.predict(X_img_2D), dtype=object)
selected_bands = alg_Ranking.band_indx
print("image shape: ", selected_img.shape)
print("desired bands",(i+1)*factor, "Total bands" , selected_bands.shape,"\n")
b, = selected_bands.shape
print(selected_bands,"\n")
bands_GCSR[i,:b] = selected_bands